<a href="https://colab.research.google.com/github/mfernandes10/MetNumUN2023I/blob/main/Lab7/week_4_iterative_linalg_Jacobi_Seidel_minresGroup23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Grupo 23

Mario Fernandez.

Diego Vesga.

Ivan Sepulveda.



# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [16]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [17]:
A

array([[1.51915195e+01, 6.22108771e-01, 4.37727739e-01, 7.85358584e-01,
        7.79975808e-01, 2.72592605e-01, 2.76464255e-01, 8.01872178e-01,
        9.58139354e-01, 8.75932635e-01],
       [3.57817270e-01, 1.55009951e+01, 6.83462935e-01, 7.12702027e-01,
        3.70250755e-01, 5.61196186e-01, 5.03083165e-01, 1.37684496e-02,
        7.72826622e-01, 8.82641191e-01],
       [3.64885984e-01, 6.15396178e-01, 1.50753812e+01, 3.68824006e-01,
        9.33140102e-01, 6.51378143e-01, 3.97202578e-01, 7.88730143e-01,
        3.16836122e-01, 5.68098653e-01],
       [8.69127390e-01, 4.36173424e-01, 8.02147642e-01, 1.51437668e+01,
        7.04260971e-01, 7.04581308e-01, 2.18792106e-01, 9.24867629e-01,
        4.42140755e-01, 9.09315959e-01],
       [5.98092228e-02, 1.84287084e-01, 4.73552788e-02, 6.74880944e-01,
        1.55946248e+01, 5.33310163e-01, 4.33240627e-02, 5.61433080e-01,
        3.29668446e-01, 5.02966833e-01],
       [1.11894318e-01, 6.07193706e-01, 5.65944643e-01, 6.76406199e-03,
   

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [18]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [19]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [20]:
np.linalg.norm(BB)

0.36436161983015336

# Do the Jacobi iteration

In [21]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [22]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.42861287e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [23]:
def jacobi_iteration(A, b, eps = 1e-7, n_iter = 50):
  diag = np.diag(A)
  B = -A.copy()
  np.fill_diagonal(B, 0)
  inv = np.diag(1./diag_1d)
  BB = inv @ B 
  c = inv @ b
  x_0 = np.ones(n)
  for _ in range(n_iter):
    x_0 = BB @ x + c
  return x_0

In [24]:
for i in range(1, 16):
    A1 = A + np.diagflat([-i]*n)
    print(np.linalg.norm(np.diag(1./np.diag(A1))@(-A1.copy()+np.diag(np.diag(A1)))),
          np.linalg.norm(jacobi_iteration(A1, b)-np.linalg.solve(A1, b)))

0.38959181027260875 0.006748907329509937
0.4185783948614869 0.014404905186773016
0.4522284025473819 0.023173930652707595
0.4917667095178099 0.03333264055689647
0.5388887887486234 0.045262992502026825
0.5960110344093966 0.059510009720590076
0.6667001660296402 0.0768835469371436
0.7564517359241753 0.09864963390538432
0.8742017351588476 0.12692079839179277
1.0355299928250665 0.16554077621839597
1.2702850939751231 0.22239785467784945
1.6439565658213244 0.31688293273593565
2.334809111760855 0.5141531579855976
4.080768845910033 1.2768381766268402
30.715327603064885 1.7776027746926315


# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [25]:
def seidel_iteration(A, b, eps = 1e-7, n_iter = 50):
  x = np.ones(b.shape[0])
  for _ in range(n_iter):
    for i in range(b.shape[0]):
      x[i] = (b[i]-np.dot(A[i][:i], x[:i])- np.dot(A[i][i+1:], x[i+1:]))/A[i,i]
  return x

In [26]:
for i in range(1, 16):
    B1 = A + np.diagflat([-i]*n)
    print(np.linalg.norm(np.diag(1./np.diag(B1))@(-B1.copy()+np.diag(np.diag(B1)))),
          np.linalg.norm(seidel_iteration(B1, b)-np.linalg.solve(B1, b)))

0.38959181027260875 1.5540063044689707e-17
0.4185783948614869 2.303412968156487e-17
0.4522284025473819 3.3107778123195866e-17
0.4917667095178099 1.717836144195444e-17
0.5388887887486234 1.8703665918870363e-17
0.5960110344093966 2.6208106743381504e-17
0.6667001660296402 3.222105849667643e-17
0.7564517359241753 3.894444544739273e-17
0.8742017351588476 4.3610271956070115e-17
1.0355299928250665 7.521581756278068e-17
1.2702850939751231 6.691626947686432e-17
1.6439565658213244 1.0829177162645093e-16
2.334809111760855 1.1775693440128312e-16
4.080768845910033 4.10633611443092e-09
30.715327603064885 2.224327617064848e+114


# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [27]:
def minimum_residual(A, b, eps = 1e-7, n_iter = 50):
  x = np.ones(b.shape[0])
  for _ in range(n_iter):
    r = A @ x - b
    k = (r @ A @ r)/np.linalg.norm(A @ r)**2
    x = x - k*r
  return x

In [28]:
for i in range(1, 16):
    C1 = A + np.diagflat([-i]*n)
    print(np.linalg.norm(np.diag(1./np.diag(C1))@(-C1.copy()+np.diag(np.diag(C1)))),
          np.linalg.norm(minimum_residual(C1, b)-np.linalg.solve(C1, b)))

0.38959181027260875 2.224517603584333e-17
0.4185783948614869 2.32747678105055e-17
0.4522284025473819 3.7570382170898963e-17
0.4917667095178099 3.1524751290996645e-17
0.5388887887486234 1.6016862586769483e-17
0.5960110344093966 2.2882574235218375e-17
0.6667001660296402 3.309641454705087e-17
0.7564517359241753 nan
0.8742017351588476 4.4969172124985776e-17
1.0355299928250665 6.331146576060244e-17
1.2702850939751231 6.177174679671291e-17
1.6439565658213244 1.0362341316682908e-16
2.334809111760855 1.0820341707075497e-13
4.080768845910033 0.0002171791368378776
30.715327603064885 2.0818042350846104


<ipython-input-27-707485f5b6e2>:5: RuntimeWarning: invalid value encountered in double_scalars
  k = (r @ A @ r)/np.linalg.norm(A @ r)**2
